# 1. Import libraries

In [1]:
# Install the Snowpark Python package into the Python virtual environment (automatically installs the appropriate version of PyArrow)
# pip install snowflake-snowpark-python
# ! pip install "snowflake-snowpark-python[modin]"
# pip install modin

In [1]:
pip install "snowflake-snowpark-python[modin]"

Note: you may need to restart the kernel to use updated packages.


# 2. Create Snowflake snowpark sesssion 
#### with SSO through a web browser

In [1]:
# If you have configured Snowflake to use single sign-on (SSO), you can configure your client application to use browser-based SSO for authentication.
from snowflake.snowpark import Session
connection_parameters = {
   "account":"ska58151.us-east-1",
   "user":"MALLESWARI.GELLI@SYNGENTA.COM",
   "role":"FEATLK_NONPROD_DEVELOPER_ROLE",
   "database":"FEATLK_DEV",
   "schema":"FEATLK_DATA",
   "warehouse":"FEATLK_INGEST_DEV",
   "authenticator":"externalbrowser"
}
session = Session.builder.configs(connection_parameters).create()

c:\Users\s1296718\AppData\Local\anaconda3\envs\feastore\lib\site-packages\snowflake\connector\config_manager.py:351: UserWarning: Bad owner or permissions on C:\Users\s1296718\.snowflake\connections.toml
  warn(f"Bad owner or permissions on {str(filep)}{chmod_message}")


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/06219a4a-a835-44d5-afaf-3926343bfb89/saml2?SAMLRequest=lZJdb9owFIb%2FSuRdO3E%2BQIkFVFkpaya2sUKndXcmcaiLYzMfh5T9%2BpnQSN1FK%2B0ucp7Xfuz3TK6eG%2BkduQGh1RSFPkEeV6WuhNpN0f1mgVPkgWWqYlIrPkUnDuhqNgHWyAPNW%2Fuo7vjvloP13EYKaP9jilqjqGYggCrWcKC2pOv8y5JGPqEHo60utUSvIu8nGAA31hkOkQqE03u09kCDoOs6v4t9bXZBRAgJSBY46ox8GPhnd6c3%2BDAgyZl3hMNXL24fhbo8wXta2wsE9HazWeHVt%2FUGefmgeq0VtA03a26OouT3d8uLADgD2LNRGo5CvwXMGVgc%2BqB0V0u256VuDq112%2FruK6h5FUi9E%2B7mxXyKDntRLRbNMj4d6%2B6pI2v2%2BdM8H7P8ZtHA%2FudpfZv82T18Lx4K%2FlRFNyXyfgzVRudqC4CWF%2BpcqHVLJEowyXBMNmFGo5CS2E%2FS9Bfy5q5QoZjtk4N17%2BE3ojQadG21kkLx3pKMozBjCcMsjUc4SaoRZjWrcZxF4ziJt%2FU2zYJzzRG6jA7tRczsvx9kEryOv4zhV9dMMV9pKcqTt9CmYfbt4kI%2F7FdEhesepbxhQuZVZTiAK1BK3V0bzqybdmtajoLZ5d

In [2]:
print(session)

<snowflake.snowpark.session.Session: account="ska58151", role="FEATLK_NONPROD_DEVELOPER_ROLE", database="FEATLK_DEV", schema="FEATLK_DATA", warehouse="FEATLK_INGEST_DEV">


In [ ]:
# # If you have configured Snowflake to use single sign-on (SSO), you can configure your client application to use browser-based SSO for authentication.
import json
from snowflake.snowpark import Session
connection_parameters = json.load(open('connection.json'))
print(connection_parameters)
# connect to the database
session = Session.builder.configs(connection_parameters).create()


In [1]:
import modin.pandas as pd
import snowflake.snowpark.modin.plugin
from snowflake.snowpark import Session

ModuleNotFoundError: No module named 'snowflake.snowpark.modin'

## NOTE: Couldn't resolve
ModuleNotFoundError: No module named 'snowflake.snowpark.modin'

Can't i run `snowflake pandas` out side of snowflake notebooks? 

# 3. Load data

In [ ]:
from snowflake.snowpark.functions import col
import pandas as pd
import snowflake.snowpark as snowpark

# Create a Snowpark session with a default connection.
from snowflake.snowpark.session import Session
session = Session.builder.configs(connection_parameters).create()

table_name = "NYC_YELLOW_TRIPS"

def load_data(session: snowpark.Session, database:str, schema:str, table_name:str): 
    '''loads data from a table exist in active session'''  
    
    # Step 1 :- Getting data from tables (Snowflake table)  
    source_table = f"{database}.{schema}.{table_name}" 
    

    # Step2: Create a snowpark DataFrame from source table.
    snowpark_df = session.table(source_table)
    
    # Step3: Create a Snowpark pandas DataFrame from existing Snowflake table
    #snowpark_pandas_df = pd.read_snowflake(source_table)

    # Step3: Convert snowpark Dataframe into your choice (pandas df, snowpark pandas, spark df)
    # Step3a: Converting the data to pandas dataframe
    pandas_df = snowpark_df.to_pandas()

    # converting a Pandas DataFrame into a Spark DataFrame.
    
    # return snowpark_df
    return pandas_df

In [ ]:
# To print out the first 10 rows of snowpark df
import time
t0 = time.perf_counter()

snowpark_df= load_data(session,database,schema,table_name)
print(snowpark_df.show())

t1 = time.perf_counter()
print("snowpark_df printing 10 rows took:", t1-t0, "seconds")


In [ ]:

# To print out the first 10 rows of pandas df
import time
t0 = time.perf_counter()

pandas_df= load_data(session,database,schema,table_name)
print("Size of df:", pandas_df.shape)
print(pandas_df.head(5))

t1 = time.perf_counter()
print("pandas_df printing 10 rows took:", t1-t0, "seconds")

# 4. Transformations

- Features aggregated by location id and refreshed every 12 hours -> AVG_FARE_1H (float), AVG_FARE_10H (float)
- Features per trip refreshed every day - PASSENGER_COUNT, TRIP_DISTANCE, FARE_AMOUNT

## f_trip data

In [ ]:
import time
t0 = time.perf_counter()

f_trip_data = data_set[["TRIP_ID","PASSENGER_COUNT","TRIP_DISTANCE","FARE_AMOUNT"]]
print(f_trip_data.head())

t1 = time.perf_counter()
print("df.head printing took:", t1-t0, "seconds")

# and if you really want your answer in minutes:
#print(f"In minutes: {(t1-t0)/60}")



# Convert pandas df to snowpark df

In [ ]:
snowpark_df = session.create_dataframe(data=f_trip_data)
snowpark_df.head()

### f_location_data

In [ ]:
data_set.dtypes["TPEP_DROPOFF_DATETIME"]

In [ ]:
data_set["TPEP_DROPOFF_DATETIME"]

In [ ]:
data_set.dtypes["TPEP_DROPOFF_DATETIME"]

In [ ]:
data_set["reach_minutes"]= (data_set["TPEP_DROPOFF_DATETIME"]).dt.total_seconds()/60
data_set["reach_minutes"]= data_set["reach_minutes"].astype(int)
data_set.head()

In [ ]:
data_set.sort_values(by=["TPEP_DROPOFF_DATETIME"],ascending=False)[["TPEP_DROPOFF_DATETIME","DOLOCATIONID","trip_time_minutes","FARE_AMOUNT"]]


In [ ]:
AVG_FARE_1H= data_set.groupby(["DOLOCATIONID"]).apply(lambda x: "FARE_AMOUNT"]]
f_loc.head()

# Create multiple entities test

In [ ]:
import pandas as pd
df = pd.read_csv("entities_list.csv")
df

In [ ]:
import json
entities = json.load(open("entities.json", "r"))
entities

In [ ]:
for index,row in enumerate(df.values):
    print(index, row)

In [ ]:
for index,row in enumerate(df.values):
    for i in index:
        print(i)